In [ ]:
X_test_jv, y_test_jv = load_japanese_vowels(split="test", return_X_y=True)

minirocket_mv_var_pipeline.score(X_test_jv, y_test_jv)

0.9945945945945946

# rocket model config  
https://github.com/sktime/sktime/blob/main/examples/minirocket.ipynb  
dataset formatting
https://github.com/sktime/sktime/blob/main/examples/AA_datatypes_and_datasets.ipynb 

In [ ]:
# Jupiter  MacOS
# BASE_DIR = "/Users/johnhanratty/Library/CloudStorage/OneDrive-Personal/IRMA_GIT/Kaggle_SignLanguage/asl-signs"
# WORKING_DIR = BASE_DIR
# !pip install nb_black --quiet
# %load_ext lab_black

# Colab
BASE_DIR = "/content/asl-signs"   #"/content/drive/MyDrive/GaggleSignLang/asl-signs"
WORKING_DIR = "/content/asl-work"
# !pip install nb_black --quiet
# print('-----ok')
# %load_ext nb_black

# KAGGLE
# BASE_DIR = "/kaggle/input/asl-signs"
# WORKING_DIR = "/kaggle/working/"
# !pip install nb_black --quiet --root-user-action=ignore
# %load_ext lab_black

import os
import gc
import shutil
import time

import json
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

import warnings
warnings.filterwarnings(action='ignore')

LANDMARK_FILES_DIR = f'{BASE_DIR}/train_landmark_files'
TRAIN_FILE = f"{BASE_DIR}/train.csv"

FRAMES_OUT = 12  #12 #16 # 16
PTS_IN_FRAME = 115
DIMS = 3

print('done')


done


In [ ]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.datasets import (
    load_japanese_vowels,  # multivariate dataset with unequal length
)
from sktime.transformations.panel.rocket import (
    MiniRocket,
    MiniRocketMultivariate,
    MiniRocketMultivariateVariable,
)

In [ ]:
# GET FEATURE FILES FROM GDRIVE 
# COLAB ONLY - MOVE FEATURE FILES TO WORKING DIRECTORY
# 
import os
import gc
import shutil

if os.getenv("COLAB_RELEASE_TAG") and not os.path.exists(WORKING_DIR):
  !mkdir '/content/asl-work'

if os.getenv("COLAB_RELEASE_TAG"):
    shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/feature_data{FRAMES_OUT}.npy", f"{WORKING_DIR}")
    shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/feature_labels.npy", f"{WORKING_DIR}")
    shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/sign_to_prediction_index_map.json", f"{WORKING_DIR}")



In [ ]:
## ROCKET LOAD, SWAP COLUMNS & shorten DATA SETS
datax = np.load(f"{WORKING_DIR}/feature_data{FRAMES_OUT}.npy")
datay = np.load(f"{WORKING_DIR}/feature_labels.npy") 

#convert(aa, from_type="numpy3D", to_type="pd-multiindex")

# FILTER TO REDUCE DATASET SIZE 
datax = datax[datay < 40]
datay = datay[datay < 40]
print("DATAX", datax.shape, datay.shape)



DATAX (15089, 12, 345) (15089,)


In [ ]:
# FIXED SIZE MODEL
#datax = np.nan_to_num(datax, copy=False)

datax = datax.swapaxes(1,2) # swap frame with data
trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.15, random_state=42)

minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(trainx)
X_train_transform = minirocket_multi.transform(trainx)

scaler = StandardScaler(with_mean=False)
X_train_scaled_transform = scaler.fit_transform(X_train_transform)

#classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier = SGDClassifier(loss='log')
classifier.fit(X_train_scaled_transform, trainy)

X_test_transform = minirocket_multi.transform(testx)
X_test_scaled_transform = scaler.transform(X_test_transform)
classifier.score(X_test_scaled_transform, testy)


0.03975265017667844

In [ ]:
# SDG
20 0.10707964601769912


# Ridge
40 0.09717314487632508
20 0.17610619469026548


In [ ]:
# PREPOCESS-A NORMALIZ
#datax = np.load(f"{WORKING_DIR}/feature_data{FRAMES_OUT}.npy")
#datay = np.load(f"{WORKING_DIR}/feature_labels.npy") 

lips_idx = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324, 146, 91, 181, 84, 17, 314, 405, 321, 375]
data_rows = datax.shape[0]

# MAKE COPY OF DATAX1 input from file
datax = datax.reshape((data_rows, FRAMES_OUT, -1, 3))  ## copy datax

# DEFINE VIEW MATRICES 
start_lips =  0
start_left =  len(lips_idx)
start_pose =  start_left + 21
start_right = start_pose + 33
end_right =   start_right + 21

lips_3d =   datax[:, :, start_lips:start_left, :]
lefth_3d =  datax[:, :, start_left:start_pose, :]
pose_3d =   datax[:, :, start_pose:start_right,:]
righth_3d = datax[:, :, start_right:end_right, :]

print(lips_3d.shape, lefth_3d.shape, pose_3d.shape, righth_3d.shape)

def check_minmax(lips_3d, lefth_3d, pose_3d, righth_3d):
    print("lips", lips_3d.shape, "left", lefth_3d.shape, "pose", pose_3d.shape, "right", righth_3d.shape)
    print("+++++++++++++++++++++++++")
    print("lips max",  np.nanmax(lips_3d[:,:,:,0]),  np.nanmax(lips_3d[:,:,:,1]),   np.nanmax(lips_3d[:,:,:,2]))
    print("lips min",  np.nanmin(lips_3d[:,:,:,0]),  np.nanmin(lips_3d[:,:,:,1]),   np.nanmin(lips_3d[:,:,:,2]))
    print("lefth max", np.nanmax(lefth_3d[:,:,:,0]), np.nanmax(lefth_3d[:,:,:,1]),  np.nanmax(lefth_3d[:,:,:,2]))
    print("lefth min", np.nanmin(lefth_3d[:,:,:,0]), np.nanmin(lefth_3d[:,:,:,1]),  np.nanmin(lefth_3d[:,:,:,2]))
    print("pose max",  np.nanmax(pose_3d[:,:,:,0]),  np.nanmax(pose_3d[:,:,:,1]),   np.nanmax(pose_3d[:,:,:,2]))
    print("pose min",  np.nanmin(pose_3d[:,:,:,0]),  np.nanmin(pose_3d[:,:,:,1]),   np.nanmin(pose_3d[:,:,:,2]))
    print("right max", np.nanmax(righth_3d[:,:,:,0]), np.nanmax(righth_3d[:,:,:,1]),np.nanmax(righth_3d[:,:,:,2]))
    print("right min", np.nanmin(righth_3d[:,:,:,0]), np.nanmin(righth_3d[:,:,:,1]),np.nanmin(righth_3d[:,:,:,2]))

print('BEFORE')
check_minmax(lips_3d, lefth_3d, pose_3d, righth_3d)


# lips_3d =   (lips_3d  - np.nanmin(lips_3d))  /  (np.nanmax(lips_3d)  -  np.nanmin(lips_3d))
# lefth_3d =  (lefth_3d - np.nanmin(lefth_3d)) /  (np.nanmax(lefth_3d) -  np.nanmin(lefth_3d))
# pose_3d =   (pose_3d -  np.nanmin(pose_3d))  /  (np.nanmax(pose_3d)  -  np.nanmin(pose_3d))
# righth_3d = (righth_3d -np.nanmin(righth_3d)) / (np.nanmax(righth_3d) - np.nanmin(righth_3d))
# lips_3d =   normalize(lips_3d.view(lips_3d.shape[0], -1),   copy=False)
# lefth_3d =  normalize(lefth_3d.view(lefth_3d.shape[0],-1),  copy=False)
# pose_3d =   normalize(pose_3d.view(pose_3d.shape[0], -1),   copy=False) 
# righth_3d = normalize(righth_3d.view(righth_3d.shape[0], -1),copy=False)
# # print('AFTER')
check_minmax(lips_3d, lefth_3d, pose_3d, righth_3d)

# Combind Componets to datax
datax=np.concatenate((lips_3d, lefth_3d, pose_3d, righth_3d), axis=2)

MIRROR = True
if MIRROR: 
    # Mirror data on x-axis (max-value)
    lips_m = lips_3d[:,:,:,0:DIMS]
    lips_m[:,:,:,0] = np.nanmax(lips_3d[:,:,:,0]) - lips_3d[:,:,:,0]
    left_m = lefth_3d[:,:,:,0:DIMS]
    left_m[:,:,:,0] = np.nanmax(lefth_3d[:,:,:,0]) - lefth_3d[:,:,:,0]
    pose_m = pose_3d[:,:,:,0:DIMS]
    pose_m[:,:,:,0] = np.nanmax(pose_3d[:,:,:,0]) - pose_3d[:,:,:,0]
    right_m = righth_3d[:,:,:,0:DIMS]
    right_m[:,:,:,0] = np.nanmax(righth_3d[:,:,:,0]) - righth_3d[:,:,:,0]

    # NOTE: reversed and swapped position of left_m and right_m 
    # so last slot is primary hand
    datam = np.concatenate((lips_m, right_m, pose_m, left_m), axis=2)
    # datam = np.nan_to_num(datam, copy=False)
    print('DATAM', datam.shape)
    # find primary hand (Count NaNs)
    #  -check whether lefth_3d OR righth_3d has more NaNs
    cl = lefth_3d.reshape(lefth_3d.shape[0],-1)
    cr = righth_3d.reshape(lefth_3d.shape[0],-1)
    # True if righth is primary (lefth_3d has more NaNs)
    cc = np.isnan(cl).sum(axis=1) > np.isnan(cr).sum(axis=1)
    print('X_in')
    print(datax[2,6,36:43,0])
    print('M_in')
    print(datam[2,6,36:43,0])

    datax[cc,:,:,:] = datam[cc,:,:,:]
    
    print('X_out true')
    print(datax[2,6,36:43:,0])

# REPLACE NaNs
print('DATAX NANS FOR REPLACEMENT', np.isnan(datax).sum())
datax = np.nan_to_num(datax, copy=False)

# SET 3D or 2D
datax = datax[:,:,:,0:DIMS]

print("FINAL SHAPE")
print("datax", datax.shape)
print("datay", datay.shape)




(7528, 12, 40, 3) (7528, 12, 21, 3) (7528, 12, 33, 3) (7528, 12, 21, 3)
BEFORE
lips (7528, 12, 40, 3) left (7528, 12, 21, 3) pose (7528, 12, 33, 3) right (7528, 12, 21, 3)
+++++++++++++++++++++++++
lips max 0.9694988131523132 0.8877384066581726 0.06905261427164078
lips min 0.09598942846059799 0.24376612901687622 -0.09753266721963882
lefth max 1.1281251907348633 1.1485391855239868 0.4857836067676544
lefth min -0.1295747607946396 -0.05647723376750946 -0.6672265529632568
pose max 1.8655558824539185 3.170896291732788 4.705069065093994
pose min -0.8853186368942261 0.07744700461626053 -5.092772483825684
right max 1.0653653144836426 1.1561946868896484 0.3266637325286865
right min -0.14405475556850433 -0.09196367859840393 -0.7793067097663879
lips (7528, 12, 40, 3) left (7528, 12, 21, 3) pose (7528, 12, 33, 3) right (7528, 12, 21, 3)
+++++++++++++++++++++++++
lips max 0.9694988131523132 0.8877384066581726 0.06905261427164078
lips min 0.09598942846059799 0.24376612901687622 -0.09753266721963882


In [ ]:
datax.shape


(3713, 12, 115, 3)

In [ ]:
# FIXED SIZE MODEL
datax = datax[:,:,-1].swapaxes(1,2) # swap frame with data
trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.15, random_state=42)
datax.shape

minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(trainx)
X_train_transform = minirocket_multi.transform(trainx)

scaler = StandardScaler(with_mean=False)
X_train_scaled_transform = scaler.fit_transform(X_train_transform)

classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_scaled_transform, trainy)

X_test_transform = minirocket_multi.transform(testx)
X_test_scaled_transform = scaler.transform(X_test_transform)
classifier.score(X_test_scaled_transform, testy)


0.049557522123893805

# VARIABLE LENGTH

In [ ]:
## ROCKET LOAD, SWAP COLUMNS & shorten DATA SETS
datax = np.load(f"{WORKING_DIR}/feature_data{FRAMES_OUT}.npy")
datay = np.load(f"{WORKING_DIR}/feature_labels.npy") 

#convert(aa, from_type="numpy3D", to_type="pd-multiindex")

# FILTER TO REDUCE DATASET SIZE 
datax = datax[datay < 20]
datay = datay[datay < 20]
print("DATAX", datax.shape, datay.shape)

DATAX (7528, 12, 345) (7528,)


In [ ]:
#datx = datax.swapaxes(1,2) 
print(datax[1,:,:])
xx = convert(datax.swapaxes(1,2), from_type="numpy3D", to_type="pd-multiindex")
#print(datax.shape)
#sum(np.isnan(datax).any(axis=2))
xx.head(3)

[[ 0.374901    0.43319136 -0.02163902 ...         nan         nan
          nan]
 [ 0.37783843  0.43361557 -0.02406255 ...         nan         nan
          nan]
 [ 0.37891465  0.4347927  -0.02103889 ...         nan         nan
          nan]
 ...
 [ 0.35126927  0.43630415 -0.00833604 ...         nan         nan
          nan]
 [ 0.35419124  0.43491921 -0.00643055 ...         nan         nan
          nan]
 [ 0.356114    0.4335793  -0.00547505 ...         nan         nan
          nan]]


var_0     var_1     var_2     var_3     var_4  \
instances timepoints                                                     
0         0           0.406616  0.576653  0.001251  0.409173  0.571919   
          1           0.404837  0.574680  0.001101  0.407595  0.569821   
          2           0.403690  0.574373  0.004082  0.406227  0.569955   

                         var_5     var_6     var_7     var_8     var_9  ...  \
instances timepoints                                                    ...   
0         0          -0.008737  0.413975  0.566722 -0.019022  0.421334  ...   
          1          -0.009022  0.412533  0.564445 -0.019520  0.419960  ...   
          2          -0.006478  0.410928  0.564933 -0.017586  0.418288  ...   

                      var_335  var_336  var_337  var_338  var_339  var_340  \
instances timepoints                                                         
0         0               NaN      NaN      NaN      NaN      NaN      NaN   
          1               NaN      NaN      NaN      NaN      NaN      NaN   
          2               NaN      NaN      NaN      NaN      NaN      NaN   

                      var_341  var_342  var_343  var_344  
instances timepoints                                      
0         0               NaN      NaN      NaN      NaN  
          1               NaN      NaN      NaN      NaN  
          2               NaN      NaN      NaN      NaN  

[3 rows x 345 columns]

In [ ]:
# variable length
minirocket_mv_var_pipeline = make_pipeline(
    MiniRocketMultivariateVariable(
        pad_value_short_series=-10.0, random_state=42, max_dilations_per_kernel=16
    ),
    StandardScaler(with_mean=False),
    RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)),
)
print(minirocket_mv_var_pipeline)

minirocket_mv_var_pipeline.fit(trainx, trainy)


minirocket_mv_var_pipeline.score(trainx, trainy)

X_test_jv, y_test_jv = load_japanese_vowels(split="test", return_X_y=True)

minirocket_mv_var_pipeline.score(testx, testy)

In [ ]:
from sktime.datasets import (
    load_japanese_vowels,  # multivariate dataset with unequal length
)

X_train_jv, y_train_jv = load_japanese_vowels(split="train", return_X_y=True)
# lets visualize the first three voice recordings with dimension 0-11

print("number of samples training: ", X_train_jv.shape[0])
print("series length of recoding 0, dimension 5: ", X_train_jv.iloc[0, 5].shape)
print("series length of recoding 1, dimension 5: ", X_train_jv.iloc[1, 0].shape)

X_train_jv.head(3)

number of samples training:  270
series length of recoding 0, dimension 5:  (20,)
series length of recoding 1, dimension 5:  (26,)


,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11
0,0 1.860936 1 1.891651 2 1.939205 3...,0 -0.207383 1 -0.193249 2 -0.239664 3...,0 0.261557 1 0.235363 2 0.258561 3...,0 -0.214562 1 -0.249118 2 -0.291458 3...,0 -0.171253 1 -0.112890 2 -0.041053 3...,0 -0.118167 1 -0.112238 2 -0.102034 3...,0 -0.277557 1 -0.311997 2 -0.383300 3...,0 0.025668 1 -0.027122 2 0.019013 3...,0 0.126701 1 0.171457 2 0.169510 3...,0 -0.306756 1 -0.289431 2 -0.314894 3...,0 -0.213076 1 -0.247722 2 -0.227908 3...,0 0.088728 1 0.093011 2 0.074638 3...
1,0 1.303905 1 1.288280 2 1.332021 3...,0 0.067256 1 0.018672 2 -0.058744 3...,0 0.597720 1 0.631579 2 0.601928 3...,0 -0.271474 1 -0.355112 2 -0.347913 3...,0 -0.236808 1 -0.119216 2 -0.053463 3...,0 -0.411125 1 -0.434425 2 -0.421753 3...,0 -0.014826 1 -0.078036 2 -0.028479 3...,0 0.113175 1 0.178121 2 0.145073 3...,0 -0.058230 1 -0.106430 2 -0.159488 3...,0 -0.173138 1 -0.181910 2 -0.127751 3...,0 0.093058 1 0.093031 2 0.019092 3...,0 0.099247 1 0.099183 2 0.113546 3...
2,0 1.462484 1 1.309815 2 1.418207 3...,0 0.174066 1 0.120183 2 0.015721 3...,0 0.505133 1 0.503046 2 0.589994 3...,0 -0.374302 1 -0.327562 2 -0.310586 3...,0 -0.362125 1 -0.356789 2 -0.477019 3...,0 -0.400335 1 -0.445498 2 -0.367101 3...,0 -0.137429 1 -0.060423 2 -0.120849 3...,0 -0.000830 1 -0.007899 2 0.066952 3...,0 0.053888 1 0.041605 2 -0.023859 3...,0 -0.237630 1 -0.231087 2 -0.224317 3...,0 0.120636 1 0.121053 2 0.175298 3...,0 0.193254 1 0.202386 2 0.156670 3...
